状态模式——处理对象的多种状态及其相互转换

**[参考链接]**
- http://blog.csdn.net/lovelion/article/details/17517213 中状态模式部分

**状态模式(State Pattern): **允许一个对象在其内部状态改变时改变它的行为, 对象看起来似乎修改了它的类, 其别名为状态对象(Objects for States). 状态模式用于解决系统中复杂对象的状态转换以及不同状态下行为的封装问题.

**包含角色:**
- Context(环境类): 环境类又称为上下文类，它是拥有多种状态的对象
- State(抽象状态类): 它用于定义一个接口以封装与环境类的一个特定状态相关的行为
- ConcreteState(具体状态类): 实现一个与环境类的一个具体状态相关的行为

**状态转换:**
- 统一由环境类负责状态的转换
- 由具体状态类来负责状态的转换

## 银行账户的不同状态实现

In [37]:
# 环境类
class Account(object):
    def __init__(self, owner, balance=0):
        self.owner = owner
        self._balance = balance
        self.state = self._init_state()
        print '%s 开户, 初始金额 %s' % (owner, balance)

    def _init_state(self):
        return NormalState(self)

    @property
    def balance(self):
        return self._balance

    @balance.setter
    def balance(self, amount):
        self._balance = amount
        self._update_state()

    def _update_state(self):
        if self._balance >= 0:
            self.state = NormalState(self)
        elif -2000 <= self._balance < 0:
            self.state = OverdraftState(self)
        elif self._balance < -2000:
            self.state = RestrictState(self)

    def deposit(self, amount):
        print self.owner, '存款', amount
        print '-' * 20
        self.state.deposit(amount)

    def withdraw(self, amount):
        print self.owner, '取款', amount
        print '-' * 20
        self.state.withdraw(amount)

    def compute_interest(self):
        self.state.compute_interest()


# 抽象状态类
class State(object):
    def __init__(self, acc):
        self.acc = acc

    def __repr__(self):
        return '%s' % self.__class__.__name__

    __str__ = __repr__

    def deposit(self, amount):
        self.acc.balance += amount
        print '存款: ', amount
        print '账户金额: ', self.acc.balance
        print '当前状态: %s' % self.acc.state

    def withdraw(self, amount):
        self.acc.balance -= amount
        print '取款: ', amount, '成功'
        print '账户金额: ', self.acc.balance
        print '当前状态: %s' % self.acc.state

    def compute_interest(self):
        raise NotImplementedError


# 正常状态
class NormalState(State):
    def compute_interest(self):
        print '账户状态正常 无需计息'


# 透支状态
class OverdraftState(State):
    def withdraw(self, amount):
        print '取款: ', amount,
        if amount >= 5000:
            print '额度不足, 无法取款'
        else:
            self.acc.balance -= amount
            print '成功'

        print '账户金额: ', self.acc.balance
        print '当前状态: %s' % self.acc.state

    def compute_interest(self):
        print '计息中, 吓死你'


# 受限状态
class RestrictState(State):
    def withdraw(self, amount):
        print '额度不足, 无法取款'
        print '账户金额: ', self.acc.balance
        print '当前状态: %s' % self.acc.state

    def compute_interest(self):
        print '计息中, 赶快还钱'

In [40]:
acc = Account('令狐冲', 1000)

令狐冲 开户, 初始金额 1000


In [41]:
acc.deposit(1000)

令狐冲 存款 1000
--------------------
存款:  1000
账户金额:  2000
当前状态: NormalState


In [42]:
acc.withdraw(3000)

令狐冲 取款 3000
--------------------
取款:  3000 成功
账户金额:  -1000
当前状态: OverdraftState


In [44]:
acc.compute_interest()

计息中, 吓死你


In [45]:
acc.withdraw(5000)

令狐冲 取款 5000
--------------------
取款:  5000 额度不足, 无法取款
账户金额:  -1000
当前状态: OverdraftState


In [46]:
acc.withdraw(2000)

令狐冲 取款 2000
--------------------
取款:  2000 成功
账户金额:  -3000
当前状态: RestrictState


In [47]:
acc.withdraw(100)

令狐冲 取款 100
--------------------
额度不足, 无法取款
账户金额:  -3000
当前状态: RestrictState


In [48]:
acc.deposit(3000)

令狐冲 存款 3000
--------------------
存款:  3000
账户金额:  0
当前状态: NormalState
